In [1]:
import pandas as pd
import json
from sqlalchemy import create_engine

In [2]:
credentials = {}
with open('credentials.json') as f:
    cred = json.load(f)
    credentials = cred['postgresql']
engine = create_engine('postgresql+psycopg2://{}:{}@{}:{}/opendata'.format(credentials['username'], credentials['password'], credentials['host'], credentials['port']))

In [3]:
import csv
from io import StringIO

def psql_insert_copy(table, conn, keys, data_iter):
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name

        sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
            table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)

In [7]:
chunksize = 10**6
i = 1
for chunk in pd.read_csv('data/Livraison/Data/TrafficAnalytics/2018_09_07_au_09/HERE_DA_32319_00000.csv', chunksize=chunksize):
    if i > 27:
        chunk.to_sql('traffic_analytics', engine, index=False, if_exists='append', method=psql_insert_copy)
        print(chunksize*i)
    i += 1

28000000
29000000
30000000
31000000
32000000
33000000
34000000
35000000
36000000
37000000


KeyboardInterrupt: 

In [3]:
streets_angers = pd.read_sql_table('streets_angers', engine)
print(streets_angers.shape)
streets_angers.head()

C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\dialects\postgresql\base.py:2690: SAWarning: Did not recognize type 'geometry' of column 'geom'
  (attype, name))


(37494, 100)


,gid,link_id,st_name,feat_id,st_langcd,num_stnmes,st_nm_pref,st_typ_bef,st_nm_base,st_nm_suff,...,phys_lanes,ver_trans,pub_access,low_mblty,priorityrd,spd_lm_src,expand_inc,trans_area,shape_leng,geom
0,1,57146091,RUE NEUVE,751186828,FRE,2,None,RUE,NEUVE,None,...,0,Y,Y,3,Y,2,None,N,0.001172,010500000001000000010200000002000000008010A12D...
1,2,57146948,ROUTE DES ESSARTS,754100338,FRE,3,None,ROUTE DES,ESSARTS,None,...,0,Y,Y,3,Y,1,None,N,0.000513,01050000000100000001020000000500000000F8BEAD9E...
2,3,57146959,None,0,None,0,None,None,None,None,...,0,N,Y,3,N,2,None,N,0.009721,01050000000100000001020000001E000000000C7648F9...
3,4,57147000,None,0,None,0,None,None,None,None,...,0,N,Y,3,N,2,None,N,0.001159,010500000001000000010200000006000000000C6E0569...
4,5,57147006,None,0,None,0,None,None,None,None,...,0,N,Y,3,N,2,None,N,0.000061,010500000001000000010200000002000000005C2331EB...


In [10]:
streets_angers.dtypes

gid             int64
link_id         int64
st_name        object
feat_id         int64
st_langcd      object
num_stnmes      int64
st_nm_pref     object
st_typ_bef     object
st_nm_base     object
st_nm_suff     object
st_typ_aft     object
st_typ_att     object
addr_type      object
l_refaddr      object
l_nrefaddr     object
l_addrsch      object
l_addrform     object
r_refaddr      object
r_nrefaddr     object
r_addrsch      object
r_addrform     object
ref_in_id       int64
nref_in_id      int64
n_shapepnt      int64
func_class     object
speed_cat      object
fr_spd_lim      int64
to_spd_lim      int64
to_lanes        int64
from_lanes      int64
               ...   
inprocdata     object
full_geom      object
urban          object
route_type     object
dironsign      object
explicatbl     object
nameonrdsn     object
postalname     object
stalename      object
vanityname     object
junctionnm     object
exitname       object
scenic_rt      object
scenic_nm      object
fourwhldr 